In [1]:
# Dependencies and Setup
import pandas as pd
import os
import csv
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "../Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
#purchase_data.head()

## Player Count

In [2]:
#Display the total number of players
player_count = len(purchase_data["SN"].unique())
player_count = [(player_count)]

#Create a data frame to print only the total players
total_players_df = pd.DataFrame(player_count, columns=["Total Players"])

#Print data frame
total_players_df.head()

,Total Players
0,576


## Purchasing Analysis (Total)

In [3]:
#Run basic calculations to obtain number of unique items, average price, etc.
unique_items = len(purchase_data["Item Name"].unique())
price_avg = purchase_data["Price"].mean()
purchase_count = purchase_data["Purchase ID"].count()
total_rev = purchase_data["Price"].sum()

#Create a summary data frame to hold the results
summary = [(unique_items, price_avg, purchase_count, total_rev)]

summary_df = pd.DataFrame(summary, columns=["Number of Unique Items", "Average Price", "Number of Purchases", "Total Revenue"])

#Optional: give the displayed data cleaner formatting
summary_df["Average Price"] = summary_df["Average Price"].map("${:.2f}".format)
summary_df["Total Revenue"] = summary_df["Total Revenue"].map("${:.2f}".format)

#Display the summary data frame
summary_df.head()

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


## Gender Demographics

In [4]:
#Drop any duplicate user purchases
unique_users = purchase_data.drop_duplicates(["SN"])
#unique_users.head()

In [5]:
#Create a data frame to show Count and Precentage grouped by Gender
gender_df = unique_users.groupby(['Gender'])
#gender_df.head()

In [6]:
#Capture player count
player_count = gender_df["SN"].count()
#player_count

In [7]:
#Capture player Precent
player_percent = gender_df["Gender"].count()/len(unique_users)
#player_percent

In [8]:
# Create a new DataFrame using both count and percentage
gender_summary_df = pd.DataFrame({"Total Count": player_count,
                                    "Percentage of Players": player_percent})
#Clean the format
gender_summary_df["Percentage of Players"] = (gender_summary_df["Percentage of Players"] * 100).astype(float).map("{0:,.2f}%".format)

gender_summary_df.head()

,Total Count,Percentage of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


## Purchasing Analysis (Gender)

In [9]:
#Group the data frame by gender
gender_purchase = purchase_data.groupby(['Gender'])
#gender_purchase.head()

In [10]:
#Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender
g_purchase_count = gender_purchase["Purchase ID"].count()
g_price_avg = gender_purchase["Price"].mean()
total_purchase = gender_purchase["Price"].sum()
avg_purchase_user = total_purchase / player_count

In [11]:
# Create a new DataFrame purchase count, avg. purchase price, avg. purchase total per person etc. by gender
purchase_summary_df = pd.DataFrame({"Purchase Count": g_purchase_count,
                                    "Average Purchase Price": g_price_avg,
                                     "Total Purchase Value": total_purchase,
                                     "Avg Total Purchase per Person": avg_purchase_user})

#Optional: give the displayed data cleaner formatting
purchase_summary_df["Average Purchase Price"] = purchase_summary_df["Average Purchase Price"].astype(float).map("${:,.2f}".format)
purchase_summary_df["Total Purchase Value"] = purchase_summary_df["Total Purchase Value"].astype(float).map("${:,.2f}".format)
purchase_summary_df["Avg Total Purchase per Person"] = purchase_summary_df["Avg Total Purchase per Person"].astype(float).map("${:,.2f}".format)

#Display the summary data frame
purchase_summary_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

In [12]:
#Establish bins for ages
age_bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 200]

#Create labels for these bins
age_names = ["<10", "10-14", "15-19","20-24","25-29","30-34","35-39", "40+"]

#Categorize the existing players using the age bins. Hint: use pd.cut()
pd.cut(unique_users["Age"], age_bins, labels=age_names)

#Set new column in data and group by that column
unique_users["Age Groups"] = pd.cut(unique_users["Age"], age_bins, labels=age_names)
new_age_group = unique_users.groupby("Age Groups")
#new_age_group

<ipython-input-12-4579d63c9e1f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_users["Age Groups"] = pd.cut(unique_users["Age"], age_bins, labels=age_names)


In [13]:
#Calculate the numbers and percentages by age group
age_group_count = new_age_group["Age"].count()
#age_group_count

In [14]:
player_count = len(purchase_data["SN"].unique())
percentage_age_group = age_group_count / player_count
#percentage_age_group

In [15]:
#Create a summary data frame to hold the results
age_demo_df = pd.DataFrame({"Total Count": age_group_count,
                                    "Percentage of Players": percentage_age_group})

#Optional: round the percentage column to two decimal points
age_demo_df["Percentage of Players"] = (age_demo_df["Percentage of Players"] * 100).astype(float).map("{0:,.2f}%".format)


#Display Age Demographics Table
age_demo_df

,Total Count,Percentage of Players
Age Groups,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

In [16]:
#Bin the purchase_data data frame by age
purchase_data["Age Groups"] = pd.cut(purchase_data["Age"], age_bins, labels=age_names)
grouped_aged = purchase_data.groupby(['Age Groups'])
#grouped_aged.head(10)

In [17]:
#Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
purchase_count = grouped_aged["Purchase ID"].count()
avg_purchase_price = grouped_aged["Price"].mean()
total_purchase_price = grouped_aged["Price"].sum()
avg_total_peruser = total_purchase_price / age_group_count

In [18]:
#Create a summary data frame to hold the results
purchase_analysis_df = pd.DataFrame({"Purchase Count": purchase_count,
                                    "Average Purchase Price": avg_purchase_price,
                                    "Total Purchase Value": total_purchase_price,
                                    "Avg Total Purchase per Person": avg_total_peruser})

#Optional: give the displayed data cleaner formatting
purchase_analysis_df["Average Purchase Price"] = purchase_analysis_df["Average Purchase Price"].astype(float).map("${:,.2f}".format)
purchase_analysis_df["Total Purchase Value"] = purchase_analysis_df["Total Purchase Value"].astype(float).map("${:,.2f}".format)
purchase_analysis_df["Avg Total Purchase per Person"] = purchase_analysis_df["Avg Total Purchase per Person"].astype(float).map("${:,.2f}".format)


#Display the summary data frame
purchase_analysis_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Groups,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81


## Top Spenders

In [19]:
#Group the data frame by user name
top_spender = purchase_data.groupby(['SN'])
#top_spender.head()

In [20]:
#Run basic calculations to obtain the results in the table below
purchase_count = top_spender["Purchase ID"].count()
avg_purchase_price = top_spender["Price"].mean()
total_purchase_value = top_spender["Price"].sum()

In [21]:
#Create a summary data frame to hold the results
top_spender_df = pd.DataFrame({"Purchase Count": purchase_count,
                                    "Average Purchase Price": avg_purchase_price,
                                    "Total Purchase Value": total_purchase_value})
#top_spender_df.head()

In [22]:
#Sort the total purchase value column in descending order
top_spender_df.sort_values(by="Total Purchase Value", ascending=False, inplace=True)
#Optional: give the displayed data cleaner formatting
top_spender_df["Average Purchase Price"] = top_spender_df["Average Purchase Price"].astype(float).map("${:,.2f}".format)
top_spender_df["Total Purchase Value"] = top_spender_df["Total Purchase Value"].astype(float).map("${:,.2f}".format)

#Display a preview of the summary data frame
top_spender_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

In [23]:
#Retrieve the Item ID, Item Name, and Item Price columns
popular_item = purchase_data.groupby(['Item ID', 'Item Name'])
#popular_item.head()

In [24]:
#Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value
purchase_count = popular_item["Purchase ID"].count()
item_price = popular_item["Price"].mean()
total_purchase_value = popular_item["Price"].sum()

In [25]:
#Create a summary data frame to hold the results
popular_item_df = pd.DataFrame({"Purchase Count": purchase_count,
                                    "Item Price": item_price,
                                    "Total Purchase Value": total_purchase_value})
#popular_item_df.head()

In [26]:
#Sort the purchase count column in descending order
new_popular_item = popular_item_df.sort_values(by="Purchase Count", ascending=False)

#Optional: give the displayed data cleaner formatting
new_popular_item["Item Price"] = new_popular_item["Item Price"].astype(float).map("${:,.2f}".format)
new_popular_item["Total Purchase Value"] = new_popular_item["Total Purchase Value"].astype(float).map("${:,.2f}".format)

#Display a preview of the summary data frame
new_popular_item.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

In [27]:
#Sort the above table by total purchase value in descending order
most_profitable = popular_item_df.sort_values(by="Total Purchase Value", ascending=False)

#Optional: give the displayed data cleaner formatting
most_profitable["Item Price"] = most_profitable["Item Price"].astype(float).map("${:,.2f}".format)
most_profitable["Total Purchase Value"] = most_profitable["Total Purchase Value"].astype(float).map("${:,.2f}".format)

#Display a preview of the data frame
most_profitable.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
